### 1) 회기동 / 이문동 음식점 리스트 수집

- 데이터 출처: 공공데이터 포털 `서울특별시 동대문구_관내 음식점 현황`
- 전처리: 전체 동대문구 내 음식점 중 `소재지` 컬럼이 회기동 / 이문동인 데이터만 추출, 음식점이 아닌 장소 제거


+ ~~음식점 리스트 수집 할 때 네이버 지도 크롤링 하지 않고 공공데이터 사용한 이유 - 네이버 지도 웹버전은 셀레니움, BS 모두 불가, 네이버 지도 모바일 버전은 검색 결과 75개만 제공~

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotVisibleException
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
import datetime
import time
import os
from requests import get
import pandas as pd
import json

In [2]:
# 동대문구 관내 음식점 데이터 호출
res_info = pd.read_csv("./dongdaemoon.csv", engine = "python", encoding = "CP949")

# 결측치 빈문자열로 처리
res_info = res_info.fillna("")

In [3]:
# 소재지가 회기동 / 이문동인 음식점 리스트 추출
location = ["회기" , " 이문"]
hoegi = res_info[res_info["소재지(지번)"].str.contains(location[0])]
imun = res_info[res_info["소재지(지번)"].str.contains(location[1])]

hoegi_name = list(hoegi["업소명"])
imun_name = list(imun["업소명"])

In [4]:
# 음식점 리스트 전처리
# 제거해야 할 목록 - 편의점, PC방, 장례시장, 꽃집, 문구점 등 제거
def remove_not_restaurant(rest_list):
    
    keyword = ["지에스", "GS", "씨유", "CU", "미니스톱", "장례식장", "PC", "피씨", "꽃", "문구"]
    
    new_rest_list = list()
    for rest in rest_list:
        find = -1
        for key in keyword:
            if (rest.find(key) != -1):
                find = 1
        if (find == -1):
            new_rest_list.append(rest)
    return (new_rest_list)

hoegi_name = remove_not_restaurant(hoegi_name)
imun_name = remove_not_restaurant(imun_name)

print("회기동 음식점 리스트:\n ", hoegi_name, end = "\n\n")
print("이문동 음식점 리스트:\n ", imun_name)

회기동 음식점 리스트:
  ['일미기사식당', '홍콩반점(경희대점)', '노랑통닭', '노바 이탈리아노', '송가네 오소리순대국', '설', '원조할머니보쌈', '레프트커피', '미식성', '삼성루', '스카이라이브', '등촌칼국수', '아비꼬경희대점', '마리채 경희대점', '회기족발', '현대반점', '투다리', '콩불 경희대점', '라황상', '안씨네원조쭈꾸미', '거복이 식당', '지금보고싶다', '칠성포차(경희대점)', '바다내음', '전골마을', '더진국수육국밥(경희대점)', '히비야', '우리집한상', '홍학식당', '도읍지', '보드람치킨', '씽씽스넥', '깡통집', '에피소드', '유성관', '낭풍(경희대점)', '멍군집', '도모다찌', '서병장대김이병', '닭한마리칼국수', '팔당닭발', '새마당(황칠염소탕.영양탕.황칠삼계탕)', '세겹살닷컴(.COM)', '이프로의도시어부', '오바(OBAR)', '코리아 푸드스토리', '이문닭발', '명성순대국', '압구정김밥', '역전할머니맥주', '우담', '화이트캣(white cat)', '뉴델리', '생도야지세상', '더블유(W)', '아이지엔빙', '요거프레소 경희대점', '79번지국수집', '회기김밥', '불타는닭갈비', '오뚝식당', '버무리떡볶이', '용용이네떡볶이', '찌개장터', '여기가좋겠네', '썬더치킨', '푸른하늘', '포보허수아비', '한탄강메기의추억', '피지(PG)', '솔레미오', '백수씨심야식당', '골드레벨파스타&필라프', '바이브', '복가주점', '순수치킨', '치킨앤칩스 경희대점', '산타나', '시니혀니비어', '비어바켓 경희대점', '자매분식', '고황식당', '홍춘천(회기경희대점)', '형', '아지트', '민들레영토경희대점', '이찌방 이야기', '상냥한눈빛의떡볶이', '호랭이양식당', '아쿠아플러스', '경희쌈밥', '송가네 낙지', '주쿤', '로코모', '규범곱창', '아메드', '솔낭구', '솔잎깨비식당', '봉추찜닭', '레알라면', '잔월', 

In [5]:
#전체 음식점 개수
print(len(hoegi_name) + len(imun_name))

951


### 2) 회기동 / 이문동 음식점의 상세 정보, 방문자 사진 크롤링

- **데이터 출처**: `네이버 지도 모바일 버전`을 Selenium, BS4을 이용하여 크롤링
- **크롤링 방법**: 네이버에서 지정한 각 음식점의 data-cid 번호 수집 후 번호를 이용하여 음식점의 상세 페이지 크롤링
- **수집 항목**: 음식점 이름, 주소, 교통, 이용시간, 메뉴이름 리스트, 방문자 음식 사진 url


- **전처리**
    - **[시간] 컬럼**<br>
    “영업시간” 문자열 제거, 영업시간이 입력되어있지 않은 경우 “영업시간이 입력되지 않았습니다” 로 대체

    - **[메뉴] 컬럼의 메뉴 정보**
        - ()안의 정보 제거 (ex, (10인분), (국내산), (L), (HOT), (ICE), (소) 등)
        - []안의 정보 제거
        - *인, *인분 제거
        - 숫자 + 단위 제거: 숫자 + kg / 숫자 + g / 숫자 + oz / 숫자 + ml / 숫자 + pcs(p, pc) / 숫자 + cc / 등급 제거

        - `+` 로 음식이 연결되어 있는 경우, `&` 로 연결되어있는 경우 분리
        - “사이즈”, L, M, R, S, 라지, 미디엄, 미디움, 스몰, 소, 중, 대, 大, 中, 小 등 제거
        - ~~중국어 음식 이름이 추가로 표기된 경우 제거~~
        
        
- [메뉴] 칼럼 전처리 예시
         
**(전처리 전)**`['벌집껍데기 160g', ' 고기모듬 400g', ' 모소리살 120g', ' 가오리살 120g', ' 꼬들살 120g', ' 오겹살 120g']` <br>
**(전처리 후)**`['오겹살', '꼬들살', '가오리살', '모리살', '고기모듬', '벌집껍데기']`

**(전처리 전)**`['야채곱창(2인)+날치알주먹밥', ' 오돌뼈+곱창(2인)+날치알주먹밥', ' 돼지껍데기+곱창(2인)+날치알주먹밥', ' 순대+곱창(2인)+날치알주먹밥', ' 삼겹살+불곱창(2인)+날치알주먹밥', ' 막창구이(2인)+날치알주먹밥', ' 갈메기+막창(2인)+날치알주먹밥', ' 곱창전골(2인)+공기밥']`<br>
**(전처리 후)** `['순대', '막창', '불곱창', '오돌뼈', '곱창전골', '공기밥', '갈메기', '곱창', '삼겹살', '돼지껍데기', '야채곱창', '막창구이', '날치알주먹밥']`

**(전처리 전)**`['모듬 스페셜 2인 450g', ' 가브리살 180g', ' 삼겹살 180g', ' 고추장닭갈비 200g', ' 암돼지삼겹 180g', ' 목살 180g', ' 고추장삼겹 180g']`<br>
**(전처리 후)** `['고추장닭갈비', '암돼지삼겹', '모듬 스페셜', '고추장삼겹', '목살', '삼겹살', '가브리살']`

In [ ]:
# 각 음식점의 data-cid 추출하기
# data-cid: 네이버에서 자체적으로 지정한 음식점 고유 번호, 상세페이지의 url에 사용됨
cid_list = []

In [7]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.headless = True
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
options.add_argument("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15")
options.add_argument("lang=ko_KR")

def crawlCid(region, rest_name, id_list):
    driver = webdriver.Chrome("./chromedriver")
    driver.implicitly_wait(3)

    for rest in rest_name:
        search_url = "https://m.map.naver.com/search2/search.naver?query=" + region + " " + rest + "&sm=hty&style=v5"
        driver.get(search_url)
        time.sleep(3)

        try:
            link = driver.find_element_by_css_selector("a.a_item")
            id_list.append(link.get_attribute("data-cid"))
        except:
            print(rest + " 재검토")
        
    driver.close()

In [8]:
# 크롤링 제한을 피하기 위해 100개씩 그룹으로 나눠서 data-cid 크롤링
for loc in location:
    rest_name = hoegi_name if loc == "회기" else imun_name
    for i in range(0, 400, 100):
        crawlCid(loc + "동", rest_name[i:i+100], cid_list)
        time.sleep(30 * 60)
    crawlCid(loc + "동", rest_name[400:], cid_list)

콩불 경희대점 재검토
안씨네원조쭈꾸미 재검토
씽씽스넥 재검토
에피소드 재검토
서병장대김이병 재검토
새마당(황칠염소탕.영양탕.황칠삼계탕) 재검토
세겹살닷컴(.COM) 재검토
오바(OBAR) 재검토
용용이네떡볶이 재검토
포보허수아비 재검토
한탄강메기의추억 재검토
피지(PG) 재검토
솔레미오 재검토
아쿠아플러스 재검토
8번지 재검토
미스터피자(경희대점) 재검토
만원의행복 재검토
또순이순대보쌈 재검토
기리반점 재검토
씨썬쌘드 재검토
오아시스(O:asis) 재검토
더 스토리 재검토
시광리 재검토
제이비파스타 재검토
(주)신년에프앤비 재검토
두-꺼비포차1924 재검토
네네치킨회기점 재검토
화주부경희대점 재검토
시옷리을미음지읒(ㅅㄹㅁㅈ) 재검토
베나레스(경희대점) 재검토
모두랑즉석떡볶기 재검토
꼼보포차 재검토
카페타세 재검토
108장 마라탕 재검토
마미스디시(Mommy's dish) 재검토
혼다라멘 재검토
태양을담은고로케 재검토
대성소처 재검토
세시봉 재검토
88슈퍼포차 재검토
베트꽁쌀국수 재검토
마스터다이닝 재검토
마라양꼬치 재검토
아지트케이(아지트K) 재검토
블랙비어(BLACK BEER) 재검토
도쿄빙수 재검토
주식회사블로썸홈 재검토
언더파스타 재검토
세븐틴(17) 재검토
이상한나라의앨리스 재검토
찬쥐가 재검토
쭈꾸미본색 경희대점 재검토
부어케그리고까를로봉봉 재검토
비스테어·피자나폴리 재검토
서선생(先生)묵은지 재검토
버거디 재검토
MJ24불가마스파랜드 재검토
사라다방 재검토
일폐이중국전통요리 재검토
포기(foggy) 재검토
모스플레이스(MODSPLACE) 재검토
카도오 재검토
청춘텐트포차 재검토
또뽀끼야 경희대점 재검토
8.Oli(에잇올리) 재검토
신(辛)장군마라탕 재검토
커버(cover)21 재검토
정앤정 JNJ 재검토
세성당 재검토
오늘도무사히 재검토
아(A)만두여기요 재검토
다향 재검토
더 와요(The wayo) 재검토
파파이스경희의료원점 재검토
커피순(Coffee SOON) 재검토
디초콜릿커피앤드경희대점 재검토
커피숲 재검토
죽이야기 경희의료원점 재검토
커피온

In [9]:
print(cid_list)

['1325638407', '21192037', '35537477', '12994386', '97834252', '38555616', '12927133', '180768078', '37764714', '18091134', '20399584', '18092499', '20057816', '20107342', '12043371', '34472466', '11762184', '1151250150', '1770334939', '1913830064', '20986153', '1872337456', '574575034', '760247602', '37383474', '18865627', '11610428', '18091308', '11803157', '1642539250', '18091300', '31222138', '1546873647', '31543063', '19418075', '34087279', '1972193185', '37565130', '1296426946', '1325066379', '12834289', '1573115649', '13116713', '11859083', '11605696', '18864972', '31420294', '1139809003', '36663206', '32005753', '1168869814', '13055141', '1129245116', '31752052', '1809516338', '19877681', '32873203', '35092309', '38328452', '1404426440', '1365572577', '41565317', '1525776322', '31487544', '11665599', '571951650', '20973394', '1834774599', '13175373', '1057629118', '37117197', '1813213958', '19805605', '1368996295', '20557360', '251307067', '31704741', '1038303014', '928500743',

In [38]:
# 각 음식점의 상세 페이지 크롤링하기|
names, addresses, traffics, times, menu_lists, scores, num_reviews = [], [], [], [], [], [], []

In [55]:
for (i, cid) in enumerate(cid_list[475:]):
    
    #음식점의 메인 페이지 요청
    url = "https://m.place.naver.com/restaurant/" + cid + "/home"
    main_req = get(url)
    soup = BeautifulSoup(main_req.content.decode('utf-8','replace'))
    time.sleep(5)
    
    # 음식점 이름
    try:
        place_name = soup.select_one("span._3XamX").get_text()
    except:
        place_name = ""

    # 주소
    try:    
        address = soup.select_one("span._2yqUQ").get_text()
    except:
        address = ""
        
    #교통
    try:
        traffic = soup.select_one("div._2P6sT").get_text()
    except:
        traffic = ""
    
    #이용 시간
    try:
        avail = soup.select_one("ul._6aUG7 > li:nth-child(2)").get_text()
    except:
        avail = ""

    #메뉴 리스트 페이지 요청
    menu_url = url.replace("/home", "") + "/menu/list"
    menu_req = get(menu_url)
    menu_soup = BeautifulSoup(menu_req.content.decode("utf-8", "replace"))
    time.sleep(5)

    #메뉴 리스트 크롤링
    try:
        menu_list = menu_soup.select("span._3yfZ1")
        for (idx, menu) in enumerate(menu_list):
            menu_list[idx] = menu.get_text()
    except:
        menu_list = []
        
    #리뷰 페이지 요청
    review_url = url.replace("/home", "") + "/review/visitor"
    review_req = get(review_url)
    review_soup = BeautifulSoup(review_req.content.decode("utf-8", "replace"))
    time.sleep(3)

    #평점
    try:
        score = review_soup.select_one("div.hRJcF").get_text()
    except:
        score = ""
        
    #리뷰수 
    try:
        num_review = review_soup.select_one("div._1XXGh").get_text()
    except:
        num_review = ""
        
    #리스트 저장
    names.append(place_name)
    addresses.append(address)
    traffics.append(traffic)
    times.append(avail)
    scores.append(score)
    num_reviews.append(num_review)
    menu_lists.append(menu_list)

    #지연 
    print(str(i) + "개" + " 완료") 
    if ((i >= 100) & (i % 100 == 0)):
        time.sleep(30 * 60)

0개 완료
1개 완료
2개 완료
3개 완료
4개 완료
5개 완료
6개 완료
7개 완료
8개 완료
9개 완료
10개 완료
11개 완료
12개 완료
13개 완료
14개 완료
15개 완료
16개 완료
17개 완료
18개 완료
19개 완료
20개 완료
21개 완료
22개 완료
23개 완료
24개 완료
25개 완료
26개 완료
27개 완료
28개 완료
29개 완료
30개 완료
31개 완료
32개 완료
33개 완료
34개 완료
35개 완료
36개 완료
37개 완료
38개 완료
39개 완료
40개 완료
41개 완료
42개 완료
43개 완료
44개 완료
45개 완료
46개 완료
47개 완료
48개 완료
49개 완료
50개 완료
51개 완료
52개 완료
53개 완료
54개 완료
55개 완료
56개 완료
57개 완료
58개 완료
59개 완료
60개 완료
61개 완료
62개 완료
63개 완료
64개 완료
65개 완료
66개 완료
67개 완료
68개 완료
69개 완료
70개 완료
71개 완료
72개 완료
73개 완료
74개 완료
75개 완료
76개 완료
77개 완료
78개 완료
79개 완료
80개 완료
81개 완료
82개 완료
83개 완료
84개 완료
85개 완료
86개 완료
87개 완료
88개 완료
89개 완료
90개 완료
91개 완료
92개 완료
93개 완료
94개 완료
95개 완료
96개 완료
97개 완료
98개 완료
99개 완료
100개 완료
101개 완료
102개 완료
103개 완료
104개 완료
105개 완료
106개 완료
107개 완료
108개 완료
109개 완료
110개 완료
111개 완료
112개 완료
113개 완료
114개 완료
115개 완료
116개 완료
117개 완료
118개 완료
119개 완료
120개 완료
121개 완료
122개 완료
123개 완료
124개 완료
125개 완료
126개 완료
127개 완료
128개 완료
129개 완료
130개 완료
131개 완료
132개 완료
133개 완료
134개 완료
135개 완료
136개 완료
137개 완료
138개 완

In [59]:
# 음식 사진 크롤링

photos = []

for cid in cid_list:
    
    photo = []
    
    try:
        driver = webdriver.Chrome("./chromedriver")
        driver.implicitly_wait(3)

        # 음식점의 사진 페이지 열기
        url = "https://m.place.naver.com/restaurant/" + cid + "/photo?filterType=음식"
        driver.get(url)
        time.sleep(3)

        prev_height = driver.execute_script("return document.body.scrollHeight")

        # 무한 스크롤
        while True:
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
            time.sleep(2)
            curr_height = driver.execute_script("return document.body.scrollHeight")
            if(curr_height == prev_height):
                break
            else:
                prev_height = driver.execute_script("return document.body.scrollHeight")

        #사진 url 수집
        imgs = driver.find_elements_by_css_selector("li img")
        for img in imgs:
            photo.append(img.get_attribute("src"))
    except:
        pass
    
    photos.append(photo)
    driver.close()

In [62]:
len(photos)

722

In [74]:
#데이터 프레임으로 저장
df = pd.DataFrame()
df["식당명"] = names
df["주소"] = addresses
df["교통"] = traffics
df["시간"] = times
df["메뉴"] = menu_lists
df["평점"] = scores
df["리뷰수"] = num_reviews
df["이미지"] = photos

df.head(10)

,식당명,주소,교통,시간,메뉴,평점,리뷰수,이미지
0,일미기사식당,서울 동대문구 회기로 105,경의중앙1경춘회기역1번 출구에서920m,영업시간 정보 수정이용시간을 알려주세요.추가,[],4.3/5,107개 (96명 참여),[https://search.pstatic.net/common/?autoRotate...
1,홍콩반점0410 경희대점,서울 동대문구 경희대로 12-1,경의중앙1경춘회기역1번 출구에서655m,영업시간매일 11:00 - 21:00 ※ 20:30분(마지막 주문),"[짬뽕, 짜장면, 탕수육(소), 짬뽕밥, 짜장밥, 군만두, 쟁반짜장]",4.27/5,381개 (326명 참여),[https://search.pstatic.net/common/?autoRotate...
2,노랑통닭 경희대점,서울 동대문구 경희대로1길 13 1층,경의중앙1경춘회기역1번 출구에서633m,영업시간일요일 16:00 - 02:00 매일 16:00 - 04:00 월요일 16:...,"[알싸한 마늘치킨 레귤러, 알싸한 마늘치킨 사이즈업, 엄청 큰 후라이드 치킨, 엄청...",4.27/5,181개 (158명 참여),[https://search.pstatic.net/common/?autoRotate...
3,노바 이탈리아노,서울 동대문구 회기로21길 44,경의중앙1경춘회기역1번 출구에서687m,영업시간일요일 휴무 평일 11:00 - 22:00 명절 휴무토요일 10:00 - 2...,"[안심,연어,닭고기 스테이크, 샐러드, 파스타, 피자, 하우스와인]",4.41/5,684개 (612명 참여),[https://search.pstatic.net/common/?autoRotate...
4,송가네오소리순대국,서울 동대문구 회기로13길 10,경의중앙1경춘회기역1번 출구에서661m,영업시간매일 10:00 - 23:00,"[술국, 곱창전골(소), 순대국, 오소리반반, 순대철판곱창]",4.52/5,84개 (75명 참여),[https://search.pstatic.net/common/?autoRotate...
5,설,서울 동대문구 경희대로 9,경의중앙1경춘회기역1번 출구에서633m,영업시간 정보 수정이용시간을 알려주세요.추가,"[회덮밥, 오뎅백반, 고기덮밥, 우동, 유부초밥]",4.27/5,125개 (112명 참여),[https://search.pstatic.net/common/?autoRotate...
6,원조할머니보쌈 경희점,서울 동대문구 이문로 31,경의중앙1경춘회기역1번 출구에서212m,영업시간매일 11:00 - 00:30 일요일은 11시까지 영업.,[],4.22/5,61개 (54명 참여),[https://search.pstatic.net/common/?autoRotate...
7,레프트커피 경희대점,서울 동대문구 회기로 164 1층 레프트커피,경의중앙1경춘회기역1번 출구에서322m,영업시간주말 10:00 - 21:00 평일 10:30 - 21:00,"[아보카도 브런치, 브라운치즈 크로플, 독일식 팬케이크 플레이트, 크래미 샌드위치,...",4.36/5,166개 (159명 참여),[https://search.pstatic.net/common/?autoRotate...
8,미식성 경희대점,서울 동대문구 경희대로1가길 32 2층,경의중앙1경춘회기역1번 출구에서766m,영업시간매일 10:20 - 20:40,[],4.42/5,49개 (46명 참여),[https://search.pstatic.net/common/?autoRotate...
9,삼성루,서울 동대문구 경희대로1가길 27,경의중앙1경춘회기역1번 출구에서766m,영업시간매일 00:00 - 24:00,"[짜장면, 탕수육+짜장1, 탕수육+짬뽕1, 탕수육+볶음밥1, 탕수육+쟁반짜장2인+만...",4.16/5,137개 (119명 참여),[https://search.pstatic.net/common/?autoRotate...


In [83]:
#전처리
import re

In [80]:
# 전처리 - [시간] 컬럼
df.loc[df["시간"].str.find("영업시간 정보 수정") != -1, "시간"] = "영업시간이 입력되지 않았습니다."
df.loc[df["시간"].str.find("영업시간이 입력되지 않았습니다.") == -1, "시간"] = \
df.loc[df["시간"].str.find("영업시간이 입력되지 않았습니다.") == -1, "시간"].str.replace("영업시간", "", case = False)

,식당명,주소,교통,시간,메뉴,평점,리뷰수,이미지
0,일미기사식당,서울 동대문구 회기로 105,경의중앙1경춘회기역1번 출구에서920m,영업시간이 입력되지 않았습니다.,[],4.3/5,107개 (96명 참여),[https://search.pstatic.net/common/?autoRotate...
1,홍콩반점0410 경희대점,서울 동대문구 경희대로 12-1,경의중앙1경춘회기역1번 출구에서655m,매일 11:00 - 21:00 ※ 20:30분(마지막 주문),"[짬뽕, 짜장면, 탕수육(소), 짬뽕밥, 짜장밥, 군만두, 쟁반짜장]",4.27/5,381개 (326명 참여),[https://search.pstatic.net/common/?autoRotate...
2,노랑통닭 경희대점,서울 동대문구 경희대로1길 13 1층,경의중앙1경춘회기역1번 출구에서633m,일요일 16:00 - 02:00 매일 16:00 - 04:00 월요일 16:00 -...,"[알싸한 마늘치킨 레귤러, 알싸한 마늘치킨 사이즈업, 엄청 큰 후라이드 치킨, 엄청...",4.27/5,181개 (158명 참여),[https://search.pstatic.net/common/?autoRotate...
3,노바 이탈리아노,서울 동대문구 회기로21길 44,경의중앙1경춘회기역1번 출구에서687m,일요일 휴무 평일 11:00 - 22:00 명절 휴무토요일 10:00 - 22:00,"[안심,연어,닭고기 스테이크, 샐러드, 파스타, 피자, 하우스와인]",4.41/5,684개 (612명 참여),[https://search.pstatic.net/common/?autoRotate...
4,송가네오소리순대국,서울 동대문구 회기로13길 10,경의중앙1경춘회기역1번 출구에서661m,매일 10:00 - 23:00,"[술국, 곱창전골(소), 순대국, 오소리반반, 순대철판곱창]",4.52/5,84개 (75명 참여),[https://search.pstatic.net/common/?autoRotate...


In [135]:
# 전처리 - [메뉴] 컬럼
preprocessed = list()

for m_list in list(df["메뉴"]):
    new_m_list = list()
    for menu in m_list:
        # ()안의 정보 제거
        menu = re.sub(r'\([^)]*\)', "", menu)
        
        # []안의 정보 제거
        menu = re.sub(r'\[[^)]*\]', "", menu)
        
        # *인, *인분 제거
        menu = re.sub(r'[0-9]인분', "", menu)
        menu = re.sub(r'[0-9]인당', "", menu)
        menu = re.sub(r'[0-9]인', "", menu)
        
        # 사이즈, L, M, R, S, 라지, 미디엄, 미디움, 스몰, 소, 중, 대 제거
        menu = re.sub("사이즈업|사이즈|L|M|R|S|라지|미디엄|미디움|스몰| 소 | 중 | 대 | 특대 | 소| 중| 대| 특대|Large|Medium|Small|Regular|大|中|小", "", menu)

        # 숫자 + 단위 제거: 숫자 + kg / 숫자 + g / 숫자 + oz / 숫자 + ml / 숫자 + pcs(p, pc) / 숫자 + cc / 등급 제거
        menu = re.sub(r'[0-9]+kg', "", menu)
        menu = re.sub(r'[0-9]+g', "", menu)
        menu = re.sub(r'[0-9]+oz', "", menu)
        menu = re.sub(r'[0-9]+ml', "", menu)
        #menu = re.sub(r'[0-9]/+', "", menu)
        menu = re.sub(r'[0-9]+pcs', "", menu)
        menu = re.sub(r'[0-9]+p', "", menu)
        menu = re.sub(r'[0-9]+cc', "", menu)
        menu = re.sub(r'[0-9]+개', "", menu)
        
        #숫자 제거
        menu = re.sub(r'[0-9]', "", menu)
        
        #Take out 제거
        menu = menu.replace("Take out", "")
        
        #Hot, Ice 제거
        menu = menu.replace("HOT", "").replace("Hot", "").replace("ICE", "").replace("Ice", "").replace("hot", "").replace("ice", "")
        
        # +로 음식이 연결되어있거나 &로 연결되어있는 경우 split
        if ((menu.find("+") != -1) | (menu.find("&") != -1) | (menu.find("or") != -1) | (menu.find(",") != -1)):
            menu = menu.replace("+", ",").replace("&", ",").replace("or", ",")
            
            for m_splited in menu.split(","):
                new_m_list.append(m_splited.strip())
        else:
            new_m_list.append(menu.replace(",", "").strip())
    
    #중복 제거후 더하기
    preprocessed.append(list(set(new_m_list)))

In [136]:
df["메뉴"] = preprocessed

In [137]:
df.head(20)

,식당명,주소,교통,시간,메뉴,평점,리뷰수,이미지
0,일미기사식당,서울 동대문구 회기로 105,경의중앙1경춘회기역1번 출구에서920m,영업시간이 입력되지 않았습니다.,[],4.3/5,107개 (96명 참여),[https://search.pstatic.net/common/?autoRotate...
1,홍콩반점0410 경희대점,서울 동대문구 경희대로 12-1,경의중앙1경춘회기역1번 출구에서655m,매일 11:00 - 21:00 ※ 20:30분(마지막 주문),"[군만두, 짬뽕밥, 짜장밥, 탕수육, 짜장면, 쟁반짜장, 짬뽕]",4.27/5,381개 (326명 참여),[https://search.pstatic.net/common/?autoRotate...
2,노랑통닭 경희대점,서울 동대문구 경희대로1길 13 1층,경의중앙1경춘회기역1번 출구에서633m,일요일 16:00 - 02:00 매일 16:00 - 04:00 월요일 16:00 -...,"[알싸한 마늘치킨 레귤러, 웰빙 파닭, 간바치, 알싸한 마늘치킨, 똥집 감자 튀김,...",4.27/5,181개 (158명 참여),[https://search.pstatic.net/common/?autoRotate...
3,노바 이탈리아노,서울 동대문구 회기로21길 44,경의중앙1경춘회기역1번 출구에서687m,일요일 휴무 평일 11:00 - 22:00 명절 휴무토요일 10:00 - 22:00,"[연어, 샐러드, 파스타, 하우스와인, 안심, 피자, 닭고기 스테이크]",4.41/5,684개 (612명 참여),[https://search.pstatic.net/common/?autoRotate...
4,송가네오소리순대국,서울 동대문구 회기로13길 10,경의중앙1경춘회기역1번 출구에서661m,매일 10:00 - 23:00,"[순대철판곱창, 순대국, 술국, 곱창전골, 오소리반반]",4.52/5,84개 (75명 참여),[https://search.pstatic.net/common/?autoRotate...
5,설,서울 동대문구 경희대로 9,경의중앙1경춘회기역1번 출구에서633m,영업시간이 입력되지 않았습니다.,"[오뎅백반, 우동, 회덮밥, 고기덮밥, 유부초밥]",4.27/5,125개 (112명 참여),[https://search.pstatic.net/common/?autoRotate...
6,원조할머니보쌈 경희점,서울 동대문구 이문로 31,경의중앙1경춘회기역1번 출구에서212m,매일 11:00 - 00:30 일요일은 11시까지 영업.,[],4.22/5,61개 (54명 참여),[https://search.pstatic.net/common/?autoRotate...
7,레프트커피 경희대점,서울 동대문구 회기로 164 1층 레프트커피,경의중앙1경춘회기역1번 출구에서322m,주말 10:00 - 21:00 평일 10:30 - 21:00,"[아메리카노, 아보카도 브런치, 크래미 샌드위치, 딥카라멜라떼, 브라운치즈 크로플,...",4.36/5,166개 (159명 참여),[https://search.pstatic.net/common/?autoRotate...
8,미식성 경희대점,서울 동대문구 경희대로1가길 32 2층,경의중앙1경춘회기역1번 출구에서766m,매일 10:20 - 20:40,[],4.42/5,49개 (46명 참여),[https://search.pstatic.net/common/?autoRotate...
9,삼성루,서울 동대문구 경희대로1가길 27,경의중앙1경춘회기역1번 출구에서766m,매일 00:00 - 24:00,"[짬뽕, 짜장, 만두, 탕짬면, 탕수육, 짜장면, 볶음밥, 쟁반짜장, 콜라, 닭탕수...",4.16/5,137개 (119명 참여),[https://search.pstatic.net/common/?autoRotate...


In [139]:
df.to_csv('(Data 1). 회기동_이문동 음식점 정보.csv', index = False)